In [42]:
import sqlite3
import pandas as pd
from datetime import datetime

In [3]:
conn = sqlite3.connect('contentDiscovery.db')
c = conn.cursor()

In [4]:
c.execute('select * from articles limit 10')

In [5]:
c.fetchall()

[(1, 13746, 72, 31, '2015-01-01 00:01:14.412609'),
 (2, 3761, 94, 41, '2015-01-01 00:04:11.049356'),
 (3, 17863, 30, 31, '2015-01-01 00:29:34.334229'),
 (4, 5131, 39, 2, '2015-01-01 00:43:55.907851'),
 (5, 14849, 9, 22, '2015-01-01 01:21:04.230631'),
 (6, 10638, 49, 33, '2015-01-01 02:36:18.726190'),
 (7, 946, 80, 29, '2015-01-01 02:42:22.486977'),
 (8, 16144, 57, 12, '2015-01-01 03:18:24.314824'),
 (9, 12393, 31, 43, '2015-01-01 03:30:24.880365'),
 (10, 2814, 59, 31, '2015-01-01 03:31:04.541074')]

CREATE TABLE users (
       		user_id INTEGER NOT NULL,
       		email VARCHAR NOT NULL,
       		PRIMARY KEY (user_id),
       		UNIQUE (email)
       	);
CREATE TABLE types (
       		type_id INTEGER NOT NULL,
       		name VARCHAR NOT NULL,
       		PRIMARY KEY (type_id),
       		UNIQUE (name)
       	);
CREATE TABLE topics (
       		topic_id INTEGER NOT NULL,
       		name VARCHAR NOT NULL,
       		PRIMARY KEY (topic_id),
       		UNIQUE (name)
       	);
CREATE TABLE articles (
       		article_id INTEGER NOT NULL,
       		author_id INTEGER NOT NULL,
       		topic_id INTEGER NOT NULL,
       		type_id INTEGER NOT NULL,
       		submission_time DATETIME NOT NULL,
       		PRIMARY KEY (article_id),
       		FOREIGN KEY(author_id) REFERENCES users(user_id)
       		FOREIGN KEY(topic_id) REFERENCES users(topic_id)
       		FOREIGN KEY(type_id) REFERENCES users(type_id)
       	);
CREATE TABLE email_content (
       		content_id INTEGER NOT NULL,
       		email_id INTEGER NOT NULL,
       		user_id INTEGER NOT NULL,
       		article_id INTEGER NOT NULL,
       		send_time DATETIME NOT NULL,
       		PRIMARY KEY (content_id),
       		FOREIGN KEY(user_id) REFERENCES users(user_id),
       		FOREIGN KEY(article_id) REFERENCES articles(article_id)
       	);

In [8]:
# Read sqlite query results into a pandas DataFrame
# con = sqlite3.connect("data/portal_mammals.sqlite")
df = pd.read_sql_query("SELECT * from articles limit 18", conn)

# verify that result of SQL query is stored in the dataframe
print(df.head())

conn.close()

   article_id  author_id  topic_id  type_id             submission_time
0           1      13746        72       31  2015-01-01 00:01:14.412609
1           2       3761        94       41  2015-01-01 00:04:11.049356
2           3      17863        30       31  2015-01-01 00:29:34.334229
3           4       5131        39        2  2015-01-01 00:43:55.907851
4           5      14849         9       22  2015-01-01 01:21:04.230631


In [32]:
conn = sqlite3.connect('contentDiscovery.db')
df = pd.read_sql_query("select * from email_content where email_id=2339", conn)
df

,content_id,email_id,user_id,article_id,send_time
0,26636,2339,14733,184,2015-01-02 08:28:32.530693
1,26637,2339,14733,182,2015-01-02 08:28:32.530693
2,26638,2339,14733,172,2015-01-02 08:28:32.530693
3,26639,2339,14733,52,2015-01-02 08:28:32.530693
4,26640,2339,14733,94,2015-01-02 08:28:32.530693
5,26641,2339,14733,138,2015-01-02 08:28:32.530693
6,26642,2339,14733,110,2015-01-02 08:28:32.530693
7,26643,2339,14733,81,2015-01-02 08:28:32.530693
8,26644,2339,14733,78,2015-01-02 08:28:32.530693
9,26645,2339,14733,54,2015-01-02 08:28:32.530693


In [36]:
!head access.log
!wc -l access.log

[02/Jan/2015:08:07:32] "GET /click?article_id=162&user_id=5475 HTTP/1.1" 200 4352
[02/Jan/2015:08:08:43] "GET /click?article_id=139&user_id=19550 HTTP/1.1" 200 3078
[02/Jan/2015:08:09:01] "GET /click?article_id=87&user_id=9408 HTTP/1.1" 200 2005
[02/Jan/2015:08:09:18] "GET /click?article_id=175&user_id=9408 HTTP/1.1" 200 3467
[02/Jan/2015:08:09:42] "GET /click?article_id=54&user_id=9531 HTTP/1.1" 200 7097
[02/Jan/2015:08:09:51] "GET /click?article_id=25&user_id=14306 HTTP/1.1" 200 2327
[02/Jan/2015:08:10:00] "GET /click?article_id=46&user_id=10739 HTTP/1.1" 200 2509
[02/Jan/2015:08:10:12] "GET /click?article_id=142&user_id=10325 HTTP/1.1" 200 3430
[02/Jan/2015:08:10:23] "GET /click?article_id=25&user_id=104 HTTP/1.1" 200 2327
[02/Jan/2015:08:10:41] "GET /click?article_id=90&user_id=10739 HTTP/1.1" 200 2123
 1770780 access.log


In [35]:
pd.read_sql_query("select * from email_content where user_id=10739 and article_id=90", conn)

,content_id,email_id,user_id,article_id,send_time
0,4968,437,10739,90,2015-01-02 08:08:14.580204


In [99]:
with open('access.log','r') as f:
    for l in f.readlines():
        elem = l.split(' ')
        user,article = [x.split('=')[1] for x in elem[2].split('&')]
        ts = datetime.strptime(elem[0][1:-1], '%d/%b/%Y:%H:%M:%S')
        wn = datetime.date(ts).isocalendar()[1]
        c.execute("INSERT INTO clicks VALUES ('%s','%d_%02d',%s,%s,%s,%s)" 
                  %(ts.strftime('%Y-%m-%d %H:%M:%S'), ts.year, wn, user, article, elem[-2], elem[-1]))
    conn.commit()

In [98]:
c = conn.cursor()

# Create table
c.execute('''
            CREATE TABLE clicks (
       		click_time DATETIME NOT NULL,
            week_num varchar not null,
            article_id INTEGER NOT NULL,
            user_id INTEGER NOT NULL,
            status INTEGER NOT NULL,
       		bsize INTEGER NOT NULL,       		
       		FOREIGN KEY(user_id) REFERENCES users(user_id),
       		FOREIGN KEY(article_id) REFERENCES articles(article_id)
       	)''')

In [113]:
# c.execute('drop table clicks')
# c.execute('alter table email_content add column week_num varchar')
# substr('00'||3, -2, 2)
c.execute("update email_content set week_num = substr(send_time,1,4)||'_'||substr('00'||(strftime('%W',send_time)+1),-2,2)")
conn.commit()

In [100]:
pd.read_sql_query("select count(*) as cnt, week_num from clicks group by week_num", conn)

,cnt,week_num
0,35838,2015_01
1,147464,2015_02
2,146516,2015_03
3,133022,2015_04
4,136998,2015_05
5,135298,2015_06
6,147308,2015_07
7,137616,2015_08
8,135845,2015_09
9,137000,2015_10


In [114]:
sql = 'select count(*) as cnt, week_num from email_content group by week_num'
# sql = 'select count(*) as cnt, type_id from articles group by type_id'
# sql = "select substr(send_time, 1,4)||'_'||(strftime('%W',send_time)+1) as wn from email_content limit 10"

pd.read_sql_query(sql, conn)

,cnt,week_num
0,209160,2015_01
1,883576,2015_02
2,893331,2015_03
3,805855,2015_04
4,826829,2015_05
5,834705,2015_06
6,879682,2015_07
7,853286,2015_08
8,820912,2015_09
9,839918,2015_10
